In [76]:
!pip install -q python-dotenv google-generativeai

In [28]:
import pandas as pd
import numpy as np

df = pd.read_csv('used_cars.csv')
df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"


In [38]:
df.shape

(4009, 12)

In [29]:
df.duplicated().sum()

0

In [30]:
df.isnull().sum()

brand             0
model             0
model_year        0
milage            0
fuel_type       170
engine            0
transmission      0
ext_col           0
int_col           0
accident        113
clean_title     596
price             0
dtype: int64

In [35]:
df.nunique()

brand             57
model           1898
model_year        34
milage          2818
fuel_type          7
engine          1146
transmission      62
ext_col          319
int_col          156
accident           2
clean_title        2
price           1569
dtype: int64

In [34]:
df['clean_title'].value_counts()

Yes    3413
No      596
Name: clean_title, dtype: int64

In [33]:
df['fuel_type'].fillna(df['fuel_type'].mode().iloc[0], inplace=True)
df['accident'].fillna(df['accident'].mode().iloc[0], inplace=True)

df['clean_title'].replace(np.nan, 'No', inplace=True)

#Create database

In [36]:
import sqlite3

# create database
conn = sqlite3.connect('car_db.db')

In [37]:
# create table and insert data into database
df.to_sql('cars', conn, if_exists='replace')

4009

In [39]:
# create cursor object to perform CRUD operation
cursor = conn.cursor()

In [43]:
# print top 5 records
data = cursor.execute('''select * from cars limit 5''')
for row in data:
  print(row)

(0, 'Ford', 'Utility Police Interceptor Base', 2013, '51,000 mi.', 'E85 Flex Fuel', '300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capability', '6-Speed A/T', 'Black', 'Black', 'At least 1 accident or damage reported', 'Yes', '$10,300')
(1, 'Hyundai', 'Palisade SEL', 2021, '34,742 mi.', 'Gasoline', '3.8L V6 24V GDI DOHC', '8-Speed Automatic', 'Moonlight Cloud', 'Gray', 'At least 1 accident or damage reported', 'Yes', '$38,005')
(2, 'Lexus', 'RX 350 RX 350', 2022, '22,372 mi.', 'Gasoline', '3.5 Liter DOHC', 'Automatic', 'Blue', 'Black', 'None reported', 'No', '$54,598')
(3, 'INFINITI', 'Q50 Hybrid Sport', 2015, '88,900 mi.', 'Hybrid', '354.0HP 3.5L V6 Cylinder Engine Gas/Electric Hybrid', '7-Speed A/T', 'Black', 'Black', 'None reported', 'Yes', '$15,500')
(4, 'Audi', 'Q3 45 S line Premium Plus', 2021, '9,835 mi.', 'Gasoline', '2.0L I4 16V GDI DOHC Turbo', '8-Speed Automatic', 'Glacier White Metallic', 'Black', 'None reported', 'No', '$34,999')


In [45]:
# commit all changes and close the connection
conn.commit()
conn.close()

#Create Model

In [60]:
from dotenv import load_dotenv
load_dotenv() ## load all the environemnt variables

import streamlit as st
import os

import google.generativeai as genai

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))


In [61]:
# Function To Load Google Gemini Model and provide queries as response
def get_gemini_response(question,prompt):
    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content([prompt[0], question])
    return response.text


In [62]:
# Fucntion To retrieve query from the database
def read_sql_query(sql, db):
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    cur.execute(sql)
    rows=cur.fetchall()
    conn.commit()
    conn.close()
    for row in rows:
        print(row)
    return rows


In [63]:
# brand model model_year milage fuel_type engine transmission ext_col int_col accident clean_title price

In [67]:
## Define Your Prompt
prompt=[
    """
    You are an expert in converting English questions to SQL query!
    The SQL database has the name carS and has the following columns - brand model model_year milage fuel_type engine transmission ext_col int_col accident clean_title price
    \n\nFor example,\nExample 1 - How many entries of records are present?,
    the SQL command will be something like this SELECT COUNT(*) FROM CARS ;
    \nExample 2 - Tell me the unique brands of car?,
    the SQL command will be something like this SELECT DISTINCT brand FROM CARS;
    also the sql code should not have ``` in beginning or end and sql word in output
    """
]

In [74]:
question = 'HOW MANY NUMBER OF RECORDS PRESENT IN TABLE'
response=get_gemini_response(question, prompt)
print(response)


2024-03-16 09:02:26.209 200 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 3299.04ms


SELECT COUNT(*) FROM CARS ;


In [75]:
response=read_sql_query(response,"car_db.db")
print("The Response is")
for row in response:
    print(row)

(4009,)
(4009,)
